# **Setup Inicial**

**Carregamento do projeto e dados**

- Importa bibliotecas principais
- Clona o repositório e adiciona o src ao sys.path
- Faz o pré-processamento dos dados com prepare_data()

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, recall_score, f1_score, confusion_matrix, roc_auc_score, accuracy_score, roc_curve, precision_recall_curve, average_precision_score, ConfusionMatrixDisplay
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.preprocessing import prepare_data
X_train, X_test, y_train, y_test, preprocessor = prepare_data("../Data/heart.csv")

# **Importação dos modelos e variações dos seus hiperparâmetros**

## **Baseline:** Antes de comparar vários modelos diferentes, é importante definir um baseline simples.  
Escolhemos utilizar **Logistic Regression** como baseline, principalmente porque:

- é um modelo linear simples;
- treina rápido;
- serve como referência mínima para saber se modelos mais complexos realmente valem a pena;
- é bastante usado em problemas de classificação binária na área da saúde.

**A partir daqui, todos os outros modelos serão comparados diretamente com o desempenho dele.**


##**Baseline (Modelo de Referência)**

- Cria um pipeline com pré-processamento + LogisticRegression
- Treina no treino e avalia no teste
- Calcula métricas para servir de referência (baseline)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score

print("Treinando baseline (Logistic Regression)...")

baseline_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=300))
])

baseline_pipeline.fit(X_train, y_train)

baseline_preds = baseline_pipeline.predict(X_test)
baseline_probs = baseline_pipeline.predict_proba(X_test)[:, 1]

baseline_results = {
    "modelo": "Baseline (LogReg)",
    "accuracy": accuracy_score(y_test, baseline_preds),
    "recall": recall_score(y_test, baseline_preds),
    "f1": f1_score(y_test, baseline_preds),
    "auc": roc_auc_score(y_test, baseline_probs)
}

baseline_results


## Definição dos modelos e hiperparâmetros

- Importa modelos candidatos
- Define o espaço de busca de hiperparâmetros (RandomizedSearchCV)

In [ ]:
# Importação de modelos
!pip install xgboost

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Definindo range de Hiperparametros
models_config = [
    {
        'name': 'RandomForest',
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'model__n_estimators': [100, 300, 500],        # N° de árvores
            'model__max_depth': [None, 10, 20],            # Profundidade
            'model__class_weight': ['balanced', None]      # Pesos
        }
    },
    {
        'name': 'SVM',
        'model': SVC(probability=True, random_state=42),
        'params': {
            'model__C': [0.1, 1, 10, 100],                 # Margem de erro
            'model__kernel': ['linear', 'rbf'],            # Tipo de linha
            'model__class_weight': ['balanced', None]
        }
    },
    {
        'name': 'XGBoost',
        'model': XGBClassifier(random_state=42, eval_metric='logloss'),
        'params': {
            'model__n_estimators': [100, 300],             # Árvores de reforço
            'model__learning_rate': [0.01, 0.1, 0.2],      # Velocidade de aprendizado
            'model__max_depth': [3, 5, 7],                 # Complexidade de cada árvore
            # XGBoost usa 'scale_pos_weight' para balancear classes.
            # Como não sabemos o valor exato agora, vamos testar alguns multiplicadores
            'model__scale_pos_weight': [1, 5, 10]
        }
    },
    {
        'name': 'Rede Neural (MLP)',
        'model': MLPClassifier(max_iter=1000, random_state=42), # max_iter alto para dar tempo de aprender
        'params': {
            'model__hidden_layer_sizes': [(50,), (100,), (50, 50)], # Camadas e neurônios
            'model__activation': ['relu', 'tanh'],
            'model__alpha': [0.0001, 0.05],                         # Regularização
            'model__learning_rate': ['constant', 'adaptive']        # Velocidade de aprendizado
        }
    },
    {
        'name': 'KNN',
        'model': KNeighborsClassifier(),
        'params': {
            'model__n_neighbors': [3, 5, 7, 9],  # N° de vizinhos
            'model__weights': ['uniform', 'distance'],
            'model__metric': ['euclidean', 'manhattan'] # Medição de distância
        }
    }
]

# **RandomizedSearchCV**




**Busca de hiperparâmetros com RandomizedSearchCV**

- Para cada modelo, cria um Pipeline (pré-processador + modelo)
- Executa busca aleatória com validação cruzada
- Armazena o melhor recall e os melhores parâmetros

In [ ]:
results = []
print ("Inciando RandomizedSearchCV")
for config in models_config:
    print(f"\nTreinando: {config['name']}...")

    # Cria um pipeline específico só para ess modelo
    pipe = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', config['model']) # Evita Data Leakage
        ])

    # Config. da Busca Aleatória
    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=config['params'],
        n_iter=15,             # Testa 15 combinações por modelo
        scoring='recall',      # O critério de vitória é o Recall!
        cv=5,                  # Validação cruzada de 5 dobras
        random_state=42,
        n_jobs=-1              # Usa todo o poder do processador
    )

    search.fit(X_train, y_train)

    # Salva o melhor resultado do modelo
    results.append({
        'Modelo': config['name'],
        'Melhor Recall': search.best_score_,
        'Melhores Params': search.best_params_,
        'Melhor Modelo Treinado': search.best_estimator_
    })

    print(f"  -> Melhor Recall do {config['name']}: {search.best_score_:.4f}")



# **Comparação: Baseline vs todos os modelos (usando os melhores resultados)**



## Tabela comparativa dos resultados

- Consolida baseline e resultados da busca em um DataFrame
- Ordena por Recall para facilitar comparação

In [ ]:
rows = [baseline_results.copy()]

for item in results:
    nome = item["Modelo"]
    modelo = item["Melhor Modelo Treinado"]

    preds = modelo.predict(X_test)

    if hasattr(modelo, "predict_proba"):
        probs = modelo.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, probs)
    else:
        auc = np.nan

    rows.append({
        "modelo": nome,
        "accuracy": accuracy_score(y_test, preds),
        "recall": recall_score(y_test, preds),
        "f1": f1_score(y_test, preds),
        "auc": auc
    })

df_all = pd.DataFrame(rows).sort_values(by="recall", ascending=False).reset_index(drop=True)
df_all

No gráfico observado acima, em que a comparação feita entre modelos de acordo com os escores de accuracy, recall, f1 e auc. Esse escoers são métricas de validação utilizadas para analisar o rendimento do modelo, no qual se destacando 3 dos 5 modleos analisados pelo grupo (KNN, LorReg, RandomForest), despertando assim a atenção do frupo para possiveís modelos de escolha para a solução do problema/desafio do grupo.

## Gráficos (modelos vs baseline)

Cada gráfico mostra o desempenho dos modelos e a linha que representa o baseline. Essas visualizações ajudam a comparar diretamente como cada modelo se comporta em relação à referência inicial.

**Comparação final e ranking das métricas**

- Normaliza as métricas em relação ao baseline
- Gera uma tabela final ordenada por Recall
- Seleciona o melhor modelo global


In [ ]:
metrics = ["recall", "f1", "accuracy", "auc"]

baseline_row = df_all[df_all["modelo"] == "Baseline (LogReg)"].iloc[0]
df_plot = df_all[df_all["modelo"] != "Baseline (LogReg)"].copy()

x = np.arange(len(df_plot["modelo"]))

for m in metrics:
    plt.figure(figsize=(10,4))
    plt.bar(x, df_plot[m].values)
    plt.axhline(baseline_row[m], linestyle="--", linewidth=2,
                label=f"Baseline = {baseline_row[m]:.3f}")
    plt.title(f"{m.upper()} — modelos vs baseline")
    plt.ylabel(m)
    plt.xticks(x, df_plot["modelo"], rotation=45, ha="right")
    plt.grid(axis="y", linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()

Os gráficos mostram métricas de avaliação dos modelos (XGBos, Random Forest, KNN, SVM, MLP), comparando-os utilizando uma baseline como cutoff onde os escores (Recall, Accuracy, F1 e AUC dos modelos vs baseline) comparam o desempenho de cinco modelos em um dataset desbalanceado de risco cardíaco.

Recall: Mede a fração de casos reais de risco de infarto detectados pelos modelos (sensibilidade); baseline em 0.612. Todos os modelos superam ligeiramente a baseline (~0.65-0.75), com variações pequenas; bom para priorizar detecção de positivos.
​

Accuracy: Proporção geral de previsões corretas; baseline em 0.886. Modelos atingem ~0.89-0.90, com Adaboost e RF próximos ou acima; útil em classes balanceadas, mas menos aqui devido ao desbalanceamento.
​
​

F1 Score: Média harmônica de precisão e recall, balanceando falsos positivos/negativos; baseline em 0.899. Modelos ~0.90-0.91, ideais para datasets desbalanceados como este.
​
​

AUC (ROC): Área sob a curva ROC, medindo discriminação entre classes em vários thresholds; baseline em 0.933. Modelos ~0.94-0.95, indicando excelente capacidade preditiva independentemente do corte.

# **Métrica principal: Recall**




A métrica principal escolhida foi **Recall**, porque o objetivo do projeto é reduzir ao máximo os **falsos negativos**.

Neste problema, um falso negativo significa classificar como “baixo risco” um paciente que na verdade tem risco de infarto.  
Esse tipo de erro é o mais crítico, porque pode atrasar um atendimento, um exame complementar ou uma intervenção preventiva.

O recall mede exatamente isso: **entre os pacientes que realmente são positivos (risco real), quantos o modelo conseguiu identificar**.  
Por isso, entre modelos com desempenho parecido, priorizamos o que apresenta **maior recall**.

As outras métricas (accuracy, f1 e AUC) ainda ajudam a complementar a análise, mas o critério central de escolha do melhor modelo é o recall.

In [ ]:
# dados
baseline_recall = baseline_results["recall"]
df_recall = df_all[df_all["modelo"] != "Baseline (LogReg)"][["modelo", "recall"]].copy()

# ordena por recall (pra ficar bem evidente)
df_recall = df_recall.sort_values(by="recall", ascending=True).reset_index(drop=True)

x = np.arange(len(df_recall["modelo"]))

plt.figure(figsize=(10,4))
plt.bar(x, df_recall["recall"].values)
plt.axhline(baseline_recall, linestyle="--", linewidth=2, label=f"Baseline = {baseline_recall:.3f}")

plt.title("Recall — modelos vs baseline")
plt.ylabel("recall")
plt.xticks(x, df_recall["modelo"], rotation=45, ha="right")
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.legend()
plt.tight_layout()
plt.show()

# só pra ficar explícito no output também
melhor = df_recall.iloc[-1]
print(f"Maior recall: {melhor['modelo']} ({melhor['recall']:.4f})")

Após termos feito a escolha dao Recall, foi analisado novamento o rendimento dos modelos para analisar qual deles tiveram a maior Recall. Assim vemos que o XGBoost passou do baseline de 0,912 com um escore de 0.9706.

# **Verficações de Segurança**

**Função para checar overfitting (treino vs teste)**

- Compara desempenho do modelo no treino e no teste
- Ajuda detectar sinais de overfitting (diferença grande entre métricas)

In [ ]:
def verificar_overfitting(modelo_treinado, X_teste, y_teste, recall_validacao):
    previsao_teste = modelo_treinado.predict(X_teste)
    recall_teste = recall_score(y_teste, previsao_teste)

    # Diferença entre o treino e teste
    diferenca = recall_validacao - recall_teste

    print(f"Recall na Validação (Estudo): {recall_validacao:.2%}")
    print(f"Recall no Teste (Prova Real): {recall_teste:.2%}")

    if diferenca > 0.10: # Se cair mais de 10%
        print(f"Overfitting! O modelo caiu {diferenca:.1%} no teste.")
    elif diferenca < -0.05:
        print(f"O modelo foi muito melhor no teste. Verifique os dados.")
    else:
        print(f"SUCESSO! Modelo Sólido. A performance se manteve.")

# **Cáculo Top 3 melhores**

**Ordenação e escolha do melhor modelo**

- Ordena a lista de resultados por Recall
- Seleciona o melhor estimador treinado


In [ ]:
# Ordena a lista de resultados pelo Recall (do maior para o menor)
results_sorted = sorted(results, key=lambda x: x['Melhor Recall'], reverse=True)

print("xX TOP 3 MODELOS COM MELHOR RECALL Xx")
for i, res in enumerate(results_sorted[:3]):
    print(f"\n#{i+1} LUGAR: {res['Modelo']}")
    print(f"   Recall Médio (Validação): {res['Melhor Recall']:.4f}")
    print(f"   Configuração Vencedora: {res['Melhores Params']}")

# Seleciona o campeão absoluto para usar no teste final
best_global_model = results_sorted[0]['Melhor Modelo Treinado']

Após analisarmos os resultados do Recall, foi feito uma análise dos tres modelos que se destacaram, entre eles (SVM, RandomForest e XGBoost), sendo o XGBoost o modelo vencedor tendo o maior escore do Recall médio (validação) = 0.9827.

# **Analisando o melhor modelo**

- Define variáveis auxiliares e prepara dados para gráficos de avaliação

In [ ]:
print("--- VERIFICAÇÃO DE ESTABILIDADE (OVERFITTING) ---\n")

# Verificamos novamente,porém só o melhor dessa vez
best_model = results_sorted[0]
print(f"Analisando: {best_model['Modelo']}")

# Chama a função criada acima mais uma vez só por segurança
verificar_overfitting(
    modelo_treinado=best_model['Melhor Modelo Treinado'],
    X_teste=X_test,
    y_teste=y_test,
    recall_validacao=best_model['Melhor Recall']
)
print("-" * 30)


# Matriz de Confusão





- Gera e plota matriz de confusão no conjunto de teste

In [ ]:
# Pegamos o melhor modelo
campeao = best_global_model

# Geramos a matriz
print(f"Visualizando performance do melhor modelo: {results_sorted[0]['Modelo']}")
fig, ax = plt.subplots(figsize=(8, 6))

ConfusionMatrixDisplay.from_estimator(
    campeao,
    X_test,
    y_test,
    display_labels=['Saudável', 'Risco de Infarto'],
    cmap='Blues',
    values_format='d',
    ax=ax
)

plt.title("Onde o Modelo Acertou e Errou")
plt.show()

Esta matriz de confusão mostra como o nosso modelo está lidando com o nosso dataset utilizado para fazer as predições do risco ou não de infarto. Ao analisar como o modelo fez as predições vemos que a precisão da classe 0 (saudável) vemos um leve imprecisão, onde classificou 38 das 82, dando um resultado falso positivo (Risco de Infarto), no entanto esse má classificação não causa tanto "problema" pelo fato dos pacientes estarem saudáveis. Entretanto, no classe 1 (Risco de Infarto) vemos o contrário, onde o nosso modleo conseguiu classificou muito bem tendo so 3 pacientes dos 102, classificados como saudáveis (classe 0) sendo na verdade paciente com Risco de Infarto. Mas mesmo que esse valores causam certo problema, a precisão em geral de classificar a classe 1 está na faixa de 0.9705, aceitando essa imprecisão na margem de erro do nosso modelo.

# Gráfico ROC e AUC

- Calcula probabilidades do modelo
- Plota ROC e PR para avaliar trade-off de limiar

In [ ]:
y_probs = best_global_model.predict_proba(X_test)[:, 1] # Pega as probabilidades da classe positiva

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

fpr, tpr, thresholds_roc = roc_curve(y_test, y_probs) # Cálculo o ROC
roc_auc = roc_auc_score(y_test, y_probs)

precision, recall, thresholds_pr = precision_recall_curve(y_test, y_probs)
avg_precision = average_precision_score(y_test, y_probs)

# Desenho do gráfico
ax1.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.3f})')
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') # Linha do "chute" (50%)
ax1.set_title('Capacidade de Separação (ROC)')
ax1.set_xlabel('Alarmes Falsos (Falsos Positivos)')
ax1.set_ylabel('Acertos Reais (Recall)')
ax1.legend(loc="lower right")
ax1.grid(alpha=0.3)

ax2.plot(recall, precision, color='green', lw=2, label=f'Precision-Recall (AP = {avg_precision:.3f})')
ax2.set_title('Precisão vs. Recall')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precisão')
ax2.legend(loc="lower left")
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

Esse gráfico apresenta a Curva ROC (Receiver Operating Characteristic) e a Curva Precision-Recall de um modelo preditivo, provavelmente para detecção de risco de infarto, junto com suas respectivas áreas sob a curva (AUC e AUC-PR). A Curva ROC plota taxa de verdadeiros positivos (sensibilidade/recall) contra falsos positivos (1-especificidade), avaliando a capacidade de discriminação em diferentes thresholds.
​

Curva ROC e AUC
A curva laranja mostra o desempenho do modelo, próxima à diagonal superior esquerda, indicando alta capacidade de separar classes (risco vs. sem risco). A linha azul diagonal representa desempenho aleatório (AUC=0.5).
​
​

AUC-ROC ≈ 0.95 (estimado visualmente, alinhado com gráficos prévios ~0.93-0.96), próximo de 1 (excelente), significando que o modelo distingue bem pacientes com risco de infarto.

Quanto maior a área sob a curva laranja, melhor; aqui, supera muito o aleatório.

Curva Precision-Recall e AUC-PR
A curva verde plota precisão (proporção correta entre preditos positivos) contra recall, útil em datasets desbalanceados como saúde (mais saudáveis que riscos). Começa alta em precisão e cai gradualmente.
​
​

AUC-PR ≈ 0.95 (ótimo), refletindo bom equilíbrio mesmo com poucos casos positivos; superior em cenários onde falsos positivos custam caro (ex.: exames desnecessários).

Threshold ótimo equilibra as duas, evitando overdetection.

Essas curvas confirmam o modelo como robusto para predição de doenças cardíacas, consistente com baselines e outros gráficos compartilhados.

# Considerações sobre o limiar de decisão




- Calcula F1 para diferentes limiares
- Escolhe limiar que maximiza a métrica desejada
- Mostra impacto do novo limiar

Mesmo que o limiar otimizado tenha aumentado a precisão e a acurácia,  
optamos por manter o limiar padrão (0.50). O motivo é que o objetivo principal  
é reduzir falsos negativos — ou seja, evitar que um paciente com risco real  
seja classificado como seguro.

Como o limiar otimizado reduz levemente a sensibilidade para positivos,  
a decisão final foi manter o limiar original, que favorece o maior número  
possível de detecções corretas entre os casos realmente positivos.

In [ ]:
# Pega as probalidades calculadas acima
fscore = (2 * precision * recall) / (precision + recall)

# Procura o Limiar ideal
ix = np.argmax(fscore)
best_thresh = thresholds_pr[ix]

print("xX RESULTADO DA OTIMIZAÇÃO Xx")
print(f"Limiar Padrão: 0.5000")
print(f"Melhor Limiar Encontrado: {best_thresh:.4f}")
print("-" * 30)
print(f"Neste novo ponto, o desempenho seria:")
print(f"-> Recall:    {recall[ix]:.4f} (Deixamos passar poucos)")
print(f"-> Precisão:  {precision[ix]:.4f} (Erramos menos alarmes)")

# Teste de acurácia da mudança
novas_previsoes = (y_probs >= best_thresh).astype(int)

acc_antiga = accuracy_score(y_test, best_global_model.predict(X_test))
acc_nova = accuracy_score(y_test, novas_previsoes)

print("-" * 30)
print(f"Acurácia Original (Limiar 0.5): {acc_antiga:.2%}")
print(f"Acurácia Otimizada (Limiar {best_thresh:.2f}): {acc_nova:.2%}")

if acc_nova > acc_antiga:
    print("Apesar do aumento na acurácia e na precisão com o novo limiar,")
    print("optamos por manter o limiar padrão. Nosso foco é evitar falsos negativos,")
    print("então priorizamos o ponto que garante o maior número possível de acertos")
    print("para pacientes que realmente têm risco.")
else:
    print("O limiar padrão (0.5) permanece como a opção mais equilibrada para o modelo.")

# **Transparência do Modelo**

**Importância de features (XGBoost)**

- Extrai nomes das features após one-hot/transformações
- Obtém feature_importances_ do XGBoost
- Plota Top-10 features mais relevantes

In [ ]:
modelo_xgboost = best_global_model.named_steps['model'] # Pega do pipeline a "inteligência da IA"

# Pega os dados brutos, antes do pré-processamento, usando o preprocessor treinado do melhor modelo
nomes_colunas = best_global_model.named_steps['preprocessor'].get_feature_names_out()

# Ranking de Sintomas
importancias = modelo_xgboost.feature_importances_

# Gráfico
import pandas as pd
import matplotlib.pyplot as plt

df_sintomas = pd.DataFrame({'Sintoma': nomes_colunas, 'Peso': importancias})
df_top10 = df_sintomas.sort_values(by='Peso', ascending=False).head(10)

plt.barh(df_top10['Sintoma'], df_top10['Peso'], color='red')
plt.gca().invert_yaxis()
plt.title('O que o modelo considera Risco de Infarto?')
plt.show()

Importância de Features

Gráfico de barras horizontais mostra o que o modelo (provavelmente XGBoost) usa mais para prever risco: "cat_ST_Slope" domina (~0.35), seguido de "cat_ChestPainType", "cat_ExerciseAngina", etc. Features categóricas de ECG, dor torácica e angina são cruciais; ajuda interpretar o "black box" do modelo.

# Conclusão do Modelo Final

O modelo escolhido foi o **XGBoost**, pois apresentou o **maior Recall** entre todos os modelos avaliados. Essa métrica foi definida como principal porque, no contexto de risco de infarto, o erro mais crítico é o falso negativo — quando um paciente de risco é classificado como seguro. O recall mede justamente quantos desses casos o modelo é capaz de identificar, e por isso é o critério central da escolha.

Além do recall superior, o XGBoost também mostrou bom desempenho em f1-score, AUC e estabilidade entre treino e teste, indicando que generaliza bem e não apresentou overfitting relevante. O limiar otimizado é apenas um ajuste opcional da forma como o modelo transforma probabilidades em decisões, mas não altera o modelo final escolhido.

Assim, o **XGBoost** é o modelo mais adequado para o problema, equilibrando desempenho e segurança clínica.
